In [1]:
import cudf as pd 
import os 
from databento_dbn import FIXED_PRICE_SCALE, UNDEF_PRICE

base_path = os.getcwd()
from typing import Literal
def build_symbol_df(symbol: Literal['GOOG', 'GOOGL']):
    goog_dir = "processed/GOOG" if symbol == 'GOOG' else "processed/GOOGL"
    goog_df_paths = [f"{base_path}/{goog_dir}/{file}" for file in os.listdir(f"{base_path}/{goog_dir}")]
    keep_cols = ["ts_recv", 'price']
    goog_df = cudf.read_parquet(goog_df_paths[0])
    orig_cols = goog_df.columns
    goog_df = goog_df.drop(columns=[col for col in goog_df.columns if col not in keep_cols])
    for file in goog_df_paths[1:]:
        df = cudf.read_parquet(file).drop(columns=[col for col in orig_cols if col not in keep_cols])
        goog_df = cudf.concat([goog_df, df])
    goog_df = goog_df.sort_values(by='ts_recv')
    return goog_df

# goog_df = build_symbol_df('GOOG')
# googl_df = build_symbol_df('GOOGL')

# goog_df['price'] = goog_df['price'] / FIXED_PRICE_SCALE
# googl_df['price'] = googl_df['price'] / FIXED_PRICE_SCALE

# goog_df.to_parquet(goog_path)
# googl_df.to_parquet(googl_path)

In [64]:
import os
import pandas as pd
from databento_dbn import UNDEF_PRICE, FIXED_PRICE_SCALE

In [71]:
BASE_PATH = os.getcwd()
DATA_PATH = "/".join(BASE_PATH.split('/')[:-1]) + '/data/'

GOOG_FILE_PATH = DATA_PATH + "GOOG/dbeq-basic-20231214.mbp-10.parquet"
goog_df = pd.read_parquet(GOOG_FILE_PATH) 
goog_df[goog_df["price"] == UNDEF_PRICE / FIXED_PRICE_SCALE]['price'] = 0 


GOOGL_FILE_PATH = DATA_PATH + "GOOG/dbeq-basic-20231214.mbp-10.parquet"
googl_df = pd.read_parquet(GOOGL_FILE_PATH)
googl_df[googl_df["price"] == UNDEF_PRICE / FIXED_PRICE_SCALE]['price'] = 0 

/tmp/ipykernel_110731/3496836747.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goog_df[goog_df["price"] == UNDEF_PRICE / FIXED_PRICE_SCALE]['price'] = 0
/tmp/ipykernel_110731/3496836747.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  googl_df[googl_df["price"] == UNDEF_PRICE / FIXED_PRICE_SCALE]['price'] = 0


In [72]:
goog_df.copy().head()

,ts_recv,ts_event,action,side,depth,price,size,ts_in_delta,bid_px_00,ask_px_00,...,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol
0,2023-12-14 05:21:03.613078998+00:00,2023-12-14 05:21:03.602144222+00:00,R,N,0,9.223372e+09,0,293174,9223372036854775807,9223372036854775807,...,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,GOOG
3,2023-12-14 05:21:05.149027583+00:00,2023-12-14 05:21:05.148703131+00:00,R,N,0,9.223372e+09,0,269373,9223372036854775807,9223372036854775807,...,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,GOOG
4,2023-12-14 10:33:46.583152942+00:00,2023-12-14 10:33:46.583120785+00:00,R,N,0,9.223372e+09,0,32157,9223372036854775807,9223372036854775807,...,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,GOOG
7,2023-12-14 12:25:56.772901170+00:00,2023-12-14 12:25:56.526049527+00:00,A,N,0,9.223372e+09,0,246834922,9223372036854775807,9223372036854775807,...,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,GOOG
8,2023-12-14 13:16:58.909484897+00:00,2023-12-14 13:16:58.909250792+00:00,T,N,0,1.346800e+02,10,57698,9223372036854775807,9223372036854775807,...,0,0,0,9223372036854775807,9223372036854775807,0,0,0,0,GOOG


In [73]:
def merge(dfA, dfB, time_unit: str, aggregations: dict): 

    NOT_AGGREGATED = ['ts_recv', 'ts_event', 'action', 'side', 'ts_in_delta', 'symbol']

    for col in dfA.columns:
        if col not in aggregations and col not in NOT_AGGREGATED:
            aggregations[col] = 'mean'

    time_amt = 5

    dfA = dfA.set_index(pd.DatetimeIndex(dfA['ts_recv']))
    dfB = dfB.set_index(pd.DatetimeIndex(dfB['ts_recv']))
    
    dfA = dfA.groupby(pd.Grouper(freq=time_unit)).agg(aggregations)
    dfB = dfB.groupby(pd.Grouper(freq=time_unit)).agg(aggregations)
    
    return dfA

In [74]:
x = merge(goog_df.copy(), googl_df.copy(), '1s', {})
x.head()

,depth,price,size,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,...,bid_sz_08,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09
ts_recv,,,,,,,,,,,,,,,,,,,,,
2023-12-14 05:21:03+00:00,0.0,9.223372e+09,0.0,9.223372e+18,9.223372e+18,0.0,0.0,0.0,0.0,9.223372e+18,...,0.0,0.0,0.0,0.0,9.223372e+18,9.223372e+18,0.0,0.0,0.0,0.0
2023-12-14 05:21:04+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-14 05:21:05+00:00,0.0,9.223372e+09,0.0,9.223372e+18,9.223372e+18,0.0,0.0,0.0,0.0,9.223372e+18,...,0.0,0.0,0.0,0.0,9.223372e+18,9.223372e+18,0.0,0.0,0.0,0.0
2023-12-14 05:21:06+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-14 05:21:07+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import adfuller
import numpy as np 
# Function to perform Augmented Dickey-Fuller test
def adf_test(series, title=''):
    result = adfuller(series, autolag='AIC')
    print(f'ADF Statistic for {title}: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')

# Performing ADF test on both series
subsample_percent = 0.05
goog_price = merged_df['price_goog'].to_numpy().astype(np.float16)
googl_price = merged_df['price_googl'].to_numpy().astype(np.float16)
# adf_test(goog_price, title='price_goog')
# adf_test(googl_price, title='price_googl')



In [ ]:

# Setting up data for Johansen cointegration test
data_for_test = merged_df[['price_googl', 'price_goog']].dropna()
data_for_test = data_for_test.to_numpy().astype(np.float16)

# Perform the Johansen cointegration test
# The 'det_order' parameter specifies the deterministic part of the setup; 0 means no deterministic part
# The 'k_ar_diff' specifies the lag, here using 1 for simplicity
johansen_test = coint_johansen(data_for_test, det_order=0, k_ar_diff=1)

# Display the results
eig = johansen_test.eig  # Eigenvalues
lr1 = johansen_test.lr1  # Test statistic
cvt = johansen_test.cvt  # Critical values for the test statistic at the 90%, 95%, 99% confidence levels

johansen_results = {
    "Eigenvalues": eig,
    "Test Statistic": lr1,
    "Critical Values (90%, 95%, 99%)": cvt
}

johansen_results

{'Eigenvalues': array([0.16074036, 0.09819602]),
 'Test Statistic': array([9778.90208348, 3627.97282724]),
 'Critical Values (90%, 95%, 99%)': array([[13.4294, 15.4943, 19.9349],
        [ 2.7055,  3.8415,  6.6349]])}